In [ ]:
import tensorflow as tf
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/dataset/train'

# Create datasets with a 20-80 train-validation split
train_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Normalize the pixel values
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

# Optimize for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)


Mounted at /content/drive
Found 633 files belonging to 4 classes.
Using 507 files for training.
Found 633 files belonging to 4 classes.
Using 126 files for validation.


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (VGG19, ResNet152V2, InceptionResNetV2,
                                           DenseNet201, Xception, MobileNetV2,
                                           NASNetMobile, EfficientNetB7)

def build_and_train_model(base_model, train_dataset, val_dataset, num_classes=4, model_name=""):
    # Freeze the base model
    base_model.trainable = False

    # Build the model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=15,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )

    # Evaluate
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"{model_name} - Validation Accuracy: {val_accuracy:.2f}")

    return model, history


In [ ]:
# Import necessary libraries
from tensorflow.keras.applications import (
    VGG19, ResNet152V2, InceptionResNetV2, DenseNet201,
    Xception, MobileNetV2, NASNetMobile, EfficientNetB7
)

# Define models to train
pretrained_models = [
    ("VGG19", VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("ResNet152V2", ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("InceptionResNetV2", InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("DenseNet201", DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("Xception", Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("MobileNetV2", MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("NASNetMobile", NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    ("EfficientNetB7", EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
]

# Storage for trained models and results
trained_models = {}
model_histories = {}
results = []

# Train each model
for model_name, base_model in pretrained_models:
    print(f"Training {model_name}...")
    model, history = build_and_train_model(base_model, train_dataset, val_dataset, model_name=model_name)
    trained_models[model_name] = model
    model_histories[model_name] = history
    val_loss, val_accuracy = model.evaluate(val_dataset)
    results.append((model_name, val_accuracy))


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Training VGG19...
Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 591s 33s/step - accuracy: 0.2330 - loss: 1.6096 - val_accuracy: 0.3810 - val_loss: 1.3450
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 541s 32s/step - accuracy: 0.3230 - loss: 1.3967 - val_accuracy: 0.4762 - val_loss: 1.2857
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 550s 35s/step - accuracy: 0.3592 - loss: 1.3432 - val_accuracy: 0.3016 - val_loss: 1.3200
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 520s 32s/step - accuracy: 0.4282 - loss: 1.2718 - val_accuracy: 0.5159 - val_loss: 1.2063
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 604s 35s/step - accu

In [ ]:
# Fine-tune each model
fine_tuned_results = []

for model_name, base_model in pretrained_models:
    print(f"Fine-tuning {model_name}...")
    trained_model = trained_models[model_name]
    base_model.trainable = True  # Unfreeze base model

    # Compile with lower learning rate
    trained_model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model
    fine_tune_history = trained_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,
        callbacks=[
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )

    # Evaluate fine-tuned model
    val_loss, val_accuracy = trained_model.evaluate(val_dataset)
    fine_tuned_results.append((model_name, val_accuracy))
    print(f"{model_name} - Fine-tuned Validation Accuracy: {val_accuracy:.2f}")


Fine-tuning VGG19...
Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1607s 100s/step - accuracy: 0.5556 - loss: 1.0237 - val_accuracy: 0.6190 - val_loss: 0.9316 - learning_rate: 1.0000e-05
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1534s 96s/step - accuracy: 0.6799 - loss: 0.7965 - val_accuracy: 0.6429 - val_loss: 0.8533 - learning_rate: 1.0000e-05
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1574s 96s/step - accuracy: 0.7610 - loss: 0.6400 - val_accuracy: 0.6190 - val_loss: 0.8110 - learning_rate: 1.0000e-05
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1538s 97s/step - accuracy: 0.8203 - loss: 0.4663 - val_accuracy: 0.6270 - val_loss: 0.8617 - learning_rate: 1.0000e-05
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1547s 97s/step - accuracy: 0.7958 - loss: 0.4542 - val_accuracy: 0.6587 - val_loss: 0.8405 - learning_rate: 1.0000e-05
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1569s 99s/step - accuracy: 0.8692 - loss: 0.3711 - val_accuracy: 0.6905 - val_loss: 0.8174 - learning_rate: 1.0000e-05
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━

In [ ]:
fine_tuned_results = []

for model_name, base_model in pretrained_models:
    print(f"Fine-tuning {model_name}...")
    trained_model = trained_models[model_name]
    base_model.trainable = True  # Unfreeze base model

    # Compile with lower learning rate
    trained_model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model
    fine_tune_history = trained_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,
        callbacks=[
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )

    # Evaluate fine-tuned model
    val_loss, val_accuracy = trained_model.evaluate(val_dataset)
    fine_tuned_results.append((model_name, val_accuracy))
    print(f"{model_name} - Fine-tuned Validation Accuracy: {val_accuracy:.2f}")

In [ ]:
import numpy as np

# Gather predictions from fine-tuned models
predictions = []
for model_name in trained_models:
    print(f"Predicting with {model_name}...")
    model = trained_models[model_name]
    preds = model.predict(val_dataset)
    predictions.append(preds)

# Average predictions across models
ensemble_preds = np.mean(predictions, axis=0)

# Convert ensemble predictions to class labels
final_preds = np.argmax(ensemble_preds, axis=1)

# Evaluate ensemble performance
from sklearn.metrics import accuracy_score

y_true = np.concatenate([y for x, y in val_dataset], axis=0)  # True labels
ensemble_accuracy = accuracy_score(y_true, final_preds)
print(f"Ensemble Model Accuracy: {ensemble_accuracy:.2f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications
import (VGG19, ResNet152V2, InceptionResNetV2, DenseNet201,Xception, MobileNetV2, NASNetMobile, EfficientNetB7)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# Function to plot ROC curves
def plot_roc_curve(models, val_dataset, n_classes=4):
    model_roc_data = {}

    # Iterate over each model
    for model_name, model in models:
        print(f"Evaluating {model_name}...")

        # Get true labels and predictions
        y_true = np.concatenate([y for _, y in val_dataset], axis=0)
        y_pred_proba = model.predict(val_dataset)

        # One-hot encode true labels
        y_true_bin = label_binarize(y_true, classes=range(n_classes))

        # Compute ROC curve and AUC
        fpr, tpr, roc_auc = {}, {}, {}
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Micro-average ROC curve
        fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Store results for plotting
        model_roc_data[model_name] = (fpr, tpr, roc_auc)

    # Plot ROC curves for all models
    plt.figure(figsize=(12, 10))
    colors = cycle(["blue", "red", "green", "orange", "purple", "cyan", "magenta", "brown", "grey", "pink"])
    for model_name, color in zip(model_roc_data.keys(), colors):
        fpr, tpr, roc_auc = model_roc_data[model_name]

        # Plot micro-average
        plt.plot(
            fpr["micro"], tpr["micro"], color=color, linestyle="--", lw=2,
            label=f"{model_name} (AUC = {roc_auc['micro']:.2f})"
        )

    plt.plot([0, 1], [0, 1], "k--", lw=2)  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate", fontsize=14)
    plt.ylabel("True Positive Rate", fontsize=14)
    plt.title("ROC Curve for Multiple Models", fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(alpha=0.3)
    plt.show()

# Example of training models
models_to_train = [
    ("VGG19", VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))),
    ("ResNet152V2", ResNet152V2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))),
    # Add other models here...
]

# Placeholder for trained models and histories
trained_models = []
model_histories = []

for model_name, base_model in models_to_train:
    print(f"Training {model_name}...")
    model, history = build_and_train_model(base_model, train_dataset, val_dataset, model_name=model_name)
    trained_models.append((model_name, model))
    model_histories.append(history)

# Plot Loss/Accuracy for all models
plot_loss_accuracy_graphs(model_histories, [model_name for model_name, _ in models_to_train])

# Plot ROC curve
plot_roc_curve(trained_models, val_dataset, n_classes=4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_loss_accuracy_graphs(models_history, model_names):
    """
    Plots Loss and Accuracy for multiple models in one graph.

    Arguments:
    models_history : list of histories - training histories of the models.
    model_names : list of model names - names for each model.
    """
    plt.figure(figsize=(14, 6))

    # Subplot 1: Accuracy Graph
    plt.subplot(1, 2, 1)
    for history, name in zip(models_history, model_names):
        plt.plot(history.history['accuracy'], label=f'{name} - Train')
        plt.plot(history.history['val_accuracy'], label=f'{name} - Validation')
    plt.title('Accuracy for Models')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

    # Subplot 2: Loss Graph
    plt.subplot(1, 2, 2)
    for history, name in zip(models_history, model_names):
        plt.plot(history.history['loss'], label=f'{name} - Train')
        plt.plot(history.history['val_loss'], label=f'{name} - Validation')
    plt.title('Loss for Models')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Assuming you have a list of trained models and their histories
models_history = [history_vgg19, history_resnet152v2, history_inceptionresnetv2, history_densenet201,
                  history_xception, history_mobilenetv2, history_nasnetmobile, history_efficientnetb7]
model_names = ["VGG19", "ResNet152V2", "InceptionResNetV2", "DenseNet201",
               "Xception", "MobileNetV2", "NASNetMobile", "EfficientNetB7"]

# Call the function to plot loss and accuracy graphs
plot_loss_accuracy_graphs(models_history, model_names)
